# 🛠 機械学習の基礎

機械学習の基礎を実践する。<br>

## 📚 手順概要
scikit-learn付属の乳がんのデータセットを使って、機械学習の教師あり（分類）を行う。
分析は、以下の手順で進めます。

1.   データセットの確認
2.   複数の機械学習法を用いた精度比較
3.   深層学習
4.   まとめ

### 1. データセットの確認

関連ライブラリの読み込み

In [ ]:
# データ操作や解析に使う Python ライブラリ「pandas」を pd という名前で使えるようにするための宣言
import pandas as pd

# 数値計算ライブラリ NumPy を np という名前で使えるようにする
import numpy as np

scikit-learnが公開しているデータは、sklearn.datasetsのメソッドを使って読み込むことができる。今回は、以下のurlにあるファイルを読み込む。<br>
今回使用する乳がんのデータセットは、ウィスコンシン大学の乳がん診断データセット(Wisconsin Diagnostic Breast Cancer, WDBC)と呼ばれている。<br>
このデータセットは、乳がんの診断に関連する特徴量を持つ患者のデータを含んでいます。乳がんデータセットを読み込む。<br>

In [ ]:
# データの読み込み
df_breast_cancer = pd.read_excel('https://github.com/iwata97/bioinfo/raw/refs/heads/main/breast_cancer_data.xlsx')

データの中身を確認してみる。

In [ ]:
df_breast_cancer.head()

今は、説明変数も目的変数も一緒になっている。<br>
目的変数をdf_targetに、説明変数をdf_dataに入れる。

In [ ]:
df_target = pd.DataFrame(df_breast_cancer["target"], columns=["target"])
df_data = df_breast_cancer.drop(columns=["target"])

特徴量のデータ型の確認

In [ ]:
# 各columnのデータ型を表示
print(f'{df_data.dtypes} \n')

データの中身を確認してみます。

In [ ]:
df_data

本データに含まれる特徴量は、いずれも連続値であることがわかります。
次に、統計量を確認します。

In [ ]:
# データにおける、数値データの統計量を表示
display(df_data.describe())

続いて、各特徴量の分布及び相関を確認します。

関連ライブラリの読み込み

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 特徴量の名前の取得
feature_names = df_data.columns

# 特徴量の分布を可視化
plt.figure(figsize=(12, 24))
for i, feature in enumerate(feature_names):
    plt.subplot(10, 3, i + 1)
    sns.histplot(df_data[feature], kde=True)
    plt.title(feature)
plt.tight_layout()
plt.show()

In [ ]:
# 特徴量間の相関を可視化
plt.figure(figsize=(10, 8))
sns.heatmap(df_data.corr(), cmap='coolwarm')
plt.title('Feature Correlation')
plt.show()

## 3. 複数の機械学習法を用いた精度比較
ロジスティック回帰、サポートベクターマシン（SVM）、ランダムフォレスト、Light GBMのモデルの精度を比較します。

*   決定木：樹形図を作成し、予測や検証をする分析
*   ランダムフォレスト：複数の決定木の多数決で分類を実施
*   サポートベクターマシン：2クラスの標本値と境界線の距離を基準に最適化

まず、データセットの説明変数と目的変数を分離する

In [ ]:
# 説明変数と目的変数をそれぞれ読み込み
X = df_breast_cancer.drop('target', axis=1).values
y = df_breast_cancer['target'].values.ravel()

中身の確認と形状（データ数・特徴量数）の確認

In [ ]:
print(X)
X.shape

In [ ]:
print(y)
y.shape

データを学習データセットとテストデータセットに分割する

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
# データを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

形状（データ数・特徴量数）を確認

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

正規化

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# 特徴量の正規化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

決定木、ランダムフォレスト、サポートベクターマシン（SVM）のモデルの精度を比較します。

In [ ]:
# モデル評価結果保存用リスト
results = []

決定木モデルの構築

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#学習
tree = DecisionTreeClassifier(max_depth=3, random_state=1)
tree.fit(X_train, y_train)

# テストデータで予測
y_pred = tree.predict(X_test)

# 精度を評価
tree_acc = accuracy_score(y_test, y_pred)

# 精度の保存
results.append({
    "Model": "Decision Tree",
    "Accuracy": tree_acc,
})

結果を見てみる

In [ ]:
print(results)

生成した決定木を視覚化してみる

In [ ]:
#pydotplus・graghvizで可視化
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
from IPython.display import Image

dot_data = export_graphviz(tree, filled=True, rounded = True, class_names=feature_names, feature_names=feature_names, out_file=None)

#グラフを出力
graph = graph_from_dot_data(dot_data)
Image(graph.create_png())

ランダムフォレストのモデル構築

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#学習
rand_forest = RandomForestClassifier(random_state=1)
rand_forest.fit(X_train, y_train)

# テストデータで予測
y_pred = rand_forest.predict(X_test)

# 精度を評価
rand_forest_acc = accuracy_score(y_test, y_pred)

# 精度の保存
results.append({
    "Model": "Random Forest",
    "Accuracy": rand_forest_acc
})

結果を見てみる

In [ ]:
print(results)

サポートベクターマシン（SVM）のモデル構築

In [ ]:
from sklearn.svm import SVC

# 学習
svc = SVC(kernel='linear', C=1, random_state=1)
svc.fit(X_train, y_train)

# テストデータで予測
y_pred = svc.predict(X_test)

# 精度を評価
svc_acc = accuracy_score(y_test, y_pred)

# 精度の保存
results.append({
    "Model": "Support Vector Machine",
    "Accuracy": svc_acc
})

結果を見てみる

In [ ]:
print(results)

In [ ]:
# results を DataFrame に変換
df_results = pd.DataFrame(results)

# 表示
df_results

グラフを作成

In [ ]:
import matplotlib.pyplot as plt

# モデル名、精度、標準誤差をリストに変換
models = [d["Model"] for d in results]
accuracies = [d["Accuracy"] for d in results]

# グラフの描画
plt.figure(figsize=(8, 5))
bars = plt.bar(models, accuracies, capsize=5, color='cornflowerblue', edgecolor='black')

# ラベルとスタイル
plt.ylabel("Accuracy")
plt.title("Model Comparison with 5-Fold Cross-Validation")
plt.ylim(0.8, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 値を棒の上に表示（任意）
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f"{acc:.3f}", ha='center', va='bottom')

plt.tight_layout()
plt.show()


# 4. 深層学習によるモデル構築と予測精度評価

In [ ]:
import numpy as np
import pandas as pd

from keras import models
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_breast_cancer

モデルの構築

In [ ]:
model=models.Sequential()
model.add(layers.Dense(200,activation="relu",input_dim=30))
model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

モデルとしては、これでおしまい。
に、ここではしてますが、行を足せばもっと多層にできます。
models.Sequential() こいつが、
データの線形結合➡活性化関数➡次のニューロンへ
をやってくれるアルゴリズムです。
で、順番に各層で何やるかを決めてるんですが、
model.add(layers.Dense(200,activation=”relu”,input_dim=30))
最初のこれが入力層、
・200個のニューロン
・活性化関数はRELU
・入力データの次元は30
だよって決めてます。
model.add(layers.Dense(100,activation=”relu”))
ほんでこれが、中間層で、
・100個のニューロン
・活性化関数はRELU
として、出力層として
model.add(layers.Dense(1,activation=”sigmoid”))
1個の出力を、シグモイド関数でだしてね、2値だから。
としているわけです。

モデルの概要を見てみる

In [ ]:
model.summary()

こんなですよと。
計26,401個のパラメータがあるよと、
各層でデータを線形結合する際の、各係数の数とその総和を教えてくれます。
モデルとしては、こうなんですが、
どうやって最適化していくの？とか、損失関数なんですか？とか、評価指標は何？
とうことを別途決めてあげないといけません。

In [ ]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

最適化アルゴリズムはadam,
2値分類問題なので、損失関数は、バイナリーの交差エントロピー、
同じく2値分類問題なので、評価指標は正解率、
としています。
最適化アルゴリズムについては、ここがわかりやすかったです。

In [ ]:
callbacks=[EarlyStopping(monitor="val_accuracy",patience=3)]

ほんでもって、検証データの正解率が、3回向上しなかったら学習終わって、を定義しといて、

In [ ]:
model.fit(X_train,
                  y_train,
                  epochs=50,
                  batch_size=100,
                  verbose=1,
                  callbacks=callbacks)

こないして、学習です。
batch_size=100
これで、1回の学習で使うデータの数を、
epochs=50
これで学習の回数、を決めています。

ここで止まりました。
構築したモデルをテストデータに適用します

In [ ]:
accuracy = model.evaluate(X_test,y_test)

# 精度の保存
results.append({
    "Model": "Deep Neural Network",
    "Accuracy": accuracy[1],
    "StdErr": 0
})

In [ ]:
# results を DataFrame に変換
df_results = pd.DataFrame(results)

# 表示
df_results

グラフにしてみる

In [ ]:
# モデル名、精度、標準誤差をリストに変換
models = [d["Model"] for d in results]
accuracies = [d["Accuracy"] for d in results]
std_errors = [d.get("StdErr", 0) for d in results]  # StdErrがない場合は0

# グラフの描画
plt.figure(figsize=(8, 5))
bars = plt.bar(models, accuracies, yerr=std_errors, capsize=5, color='cornflowerblue', edgecolor='black')

# ラベルとスタイル
plt.ylabel("Accuracy")
plt.title("Model Comparison with 5-Fold Cross-Validation")
plt.ylim(0.8, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 値を棒の上に表示（任意）
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f"{acc:.3f}", ha='center', va='bottom')

plt.tight_layout()
plt.show()


## 5. まとめ

アルゴリズムの中では、パラメータチューニングありのSVMが最も高い精度で乳がん患者を予測することができていました。

# 参考Webページ

### Pythonを用いた乳がんデータセットの分析と予測
https://qiita.com/mahomaho/items/5e0e3fcf7d4dc881d9cd

### Scikit-learn乳がん診断データセットで深層ニューラルネットワークしてみる
https://banga-heavy.com/scikit-learn%E4%B9%B3%E3%81%8C%E3%82%93%E8%A8%BA%E6%96%AD%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88%E3%81%A7%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/